#ECSE 415 Assignment 3 Part 1
Hanwen Wang 260778557

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# make path = './' connect to the path of the folder stroing images
path = '/content/drive/My Drive/ECSE_415_F_2020/Assignment_3/'

Mounted at /content/drive


# 1. Image Classification using RF and SVM

## 1.1

In [22]:
from sklearn import svm
from sklearn import datasets
import matplotlib.pyplot as plt
import numpy as np
import cv2

def HoG_feature(input_images):
  img_size = (64, 64) # h x w in pixels
  cell_size = (8, 8)  # h x w in pixels
  block_size = (4, 4)  # h x w in cells
  nbins = 4  # number of orientation bins

  # create HoG Object
  # winSize is the size of the image cropped to multiple of the cell size
  # all arguments should be given in terms of number of pixels
  hog = cv2.HOGDescriptor(_winSize=(img_size[1] // cell_size[1] * cell_size[1],
                    img_size[0] // cell_size[0] * cell_size[0]),
                _blockSize=(block_size[1] * cell_size[1],block_size[0] * cell_size[0]),
                _blockStride=(cell_size[1],cell_size[0]),
                _cellSize=(cell_size[1], cell_size[0]),
                _nbins=nbins)

  features = []
  for i in range(input_images.shape[0]):
      features.append(hog.compute(input_images[i].reshape(64,64).astype(np.uint8)).reshape(1, -1))
  features = np.vstack(features)
  return features

#load train images and test images
train_images_3D=np.load(path+'flower_subset.npz')['train_images'] #shape(1556,128,128)
test_images_3D=np.load(path+'flower_subset.npz')['test_images']  #shape(90,128,128)

#define width and height
width=64
height=64
dim=(width,height)

#define two 3 dimentional arrays filled with zeros
resized_train_image_3D=np.zeros((len(train_images_3D),width,height))
resized_test_image_3D=np.zeros((len(test_images_3D),width,height))

#resize train_images from (1556,128,128) to (1556,64,64)
for idx in range(len(train_images_3D)):
    img=train_images_3D[idx,:,:]
    img_sm=cv2.resize(img,dim,interpolation=cv2.INTER_AREA)
    resized_train_image_3D[idx,:,:]=img_sm

#resize test_images from (1556,128,128) to (1556,64,64)
for idx in range(len(test_images_3D)):
    img=test_images_3D[idx,:,:]
    img_sm=cv2.resize(img,dim,interpolation=cv2.INTER_AREA)
    resized_test_image_3D[idx,:,:]=img_sm

#reshape train_image to (1556,4096) and test_images to (90,4096)
train_images_2D=np.reshape(resized_train_image_3D,(resized_train_image_3D.shape[0],resized_train_image_3D.shape[1]*resized_train_image_3D.shape[2]))
test_images_2D=np.reshape(resized_test_image_3D,(resized_test_image_3D.shape[0],resized_test_image_3D.shape[1]*resized_test_image_3D.shape[2]))

#multiply each value in both arrays by 255 because images are in range [0,1] at first
magnified_train_images_2D=255*train_images_2D
magnified_test_images_2D=255*test_images_2D

train_features=HoG_feature(magnified_train_images_2D) #shape(1556,1600)
test_features=HoG_feature(magnified_test_images_2D) #shape(90,1600)
print("Train images output feature vector size: ",train_features.shape)
print("Test images output feature vector size: ",test_features.shape)

Train images output feature vector size:  (1556, 1600)
Test images output feature vector size:  (90, 1600)


## 1.2

In [30]:
train_labels=np.load(path+'flower_subset.npz')['train_labels']

clf = svm.SVC(gamma='auto', C=1.)
clf.fit(train_features,train_labels)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

## 1.3

In [34]:
from sklearn.metrics import accuracy_score
test_labels=np.load(path+'flower_subset.npz')['test_labels']

predicted_label=clf.predict(test_features)
print("Predicted Label:", predicted_label)
print("Actual Label:", test_labels)
print("Classification accuracy: {:.2%}".format(accuracy_score(test_labels, predicted_label)))

Predicted Label: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Actual Label: [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7
 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8]
Classification accuracy: 11.11%


## 1.4

In [42]:
#gamma = scale and C = 10000
new_clf = svm.SVC(gamma='scale', C=10000)
new_clf.fit(train_features,train_labels)

new_predicted_label=new_clf.predict(test_features)
print("Predicted Label:", new_predicted_label)
print("Actual Label:", test_labels)
print("Classification accuracy: {:.2%}".format(accuracy_score(test_labels, new_predicted_label)))

Predicted Label: [0 0 0 8 0 3 0 0 0 0 0 7 5 1 1 1 1 6 1 7 1 2 3 2 2 2 2 2 2 6 5 3 1 3 3 3 3
 1 3 6 4 4 4 4 4 4 4 4 4 4 0 2 5 6 5 5 5 1 4 5 1 1 1 7 6 6 6 1 3 6 1 7 7 1
 7 3 1 7 7 0 0 8 8 8 8 0 0 8 0 5]
Actual Label: [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7
 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8]
Classification accuracy: 61.11%


## 1.5

In [43]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=10, max_depth=5, criterion='entropy')
rf_clf.fit(train_features,train_labels)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## 1.6

In [44]:
rf_predicted_label=rf_clf.predict(test_features)
print("Predicted Label:", rf_predicted_label)
print("Actual Label:", test_labels)
print("Classification accuracy: {:.2%}".format(accuracy_score(test_labels, rf_predicted_label)))

Predicted Label: [4 4 4 4 4 6 4 7 4 4 4 4 1 1 3 4 4 6 4 1 4 2 3 4 2 2 2 2 6 4 4 3 3 3 1 2 5
 3 2 3 4 4 4 4 4 4 7 4 4 4 4 6 5 2 5 0 4 7 1 4 3 7 6 7 2 1 6 1 2 6 1 6 1 7
 7 7 1 7 4 1 4 8 8 1 8 4 0 8 4 0]
Actual Label: [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7
 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8]
Classification accuracy: 38.89%


## 1.7

In [133]:
new_rf_clf = RandomForestClassifier(n_estimators=90, max_depth=60, criterion='entropy')
new_rf_clf.fit(train_features,train_labels)

new_rf_predicted_label=new_rf_clf.predict(test_features)
print("Predicted Label:", new_rf_predicted_label)
print("Actual Label:", test_labels)
print("Classification accuracy: {:.2%}".format(accuracy_score(test_labels, new_rf_predicted_label)))

Predicted Label: [0 0 0 0 4 3 8 0 0 0 4 6 1 1 1 0 1 1 1 5 4 2 3 2 2 2 2 2 2 5 0 3 3 3 1 3 3
 1 3 6 4 4 4 4 4 4 7 4 4 4 0 2 5 1 1 4 5 1 4 7 6 1 1 4 2 6 6 1 3 6 1 1 7 1
 7 7 1 7 4 1 0 8 8 1 8 4 0 8 0 8]
Actual Label: [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3
 3 3 3 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 7 7 7 7
 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8]
Classification accuracy: 55.56%


## 1.8

From the two sets of values shown above, for non-linear SVM classifier, the classification accuracy is 61.11%, while for random forest classifier, the classification accuracy is 55.56%. Non-lenear SVM classifier provides better results.

In [135]:
random=[1,5,10,20,50]

for i in random:
  print("When random_state equals to ",i)
  new_clf_random = svm.SVC(gamma='scale', C=10000, random_state=i)
  new_clf_random.fit(train_features,train_labels)
  new_predicted_label_random=new_clf_random.predict(test_features)
  print("SVM classification accuracy: {:.2%}".format(accuracy_score(test_labels, new_predicted_label_random)))

  new_rf_clf_random = RandomForestClassifier(n_estimators=90, max_depth=60, criterion='entropy', random_state=i)
  new_rf_clf_random.fit(train_features,train_labels)
  new_rf_predicted_label_random=new_rf_clf_random.predict(test_features)
  print("RF classification accuracy: {:.2%}".format(accuracy_score(test_labels, new_rf_predicted_label_random)))
  print("\n")

When random_state equals to  1
SVM classification accuracy: 61.11%
RF classification accuracy: 47.78%


When random_state equals to  5
SVM classification accuracy: 61.11%
RF classification accuracy: 52.22%


When random_state equals to  10
SVM classification accuracy: 61.11%
RF classification accuracy: 48.89%


When random_state equals to  20
SVM classification accuracy: 61.11%
RF classification accuracy: 50.00%


When random_state equals to  50
SVM classification accuracy: 61.11%
RF classification accuracy: 53.33%




As we can see values shown above, SVM classification accuracies are always 61.11%, no matter how random_state value changes. RF classification accuracies are continuously changing around 50%. Consequently, SVM classification is more stable and robust to the change in random state in this question.